In [ ]:
local = True

In [ ]:
if not local:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [ ]:
if not local:
    %cd /content/drive/MyDrive/Sun/ML Shock/Final project/

In [ ]:
import sys
if local:
    sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")
else:
    sys.path.insert(0,"/content/drive/MyDrive/Sun/ML Shock/Final project/")

In [ ]:
# specify the image classes
classes = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]
N_classes = len(classes)

In [ ]:
# Import libraries
import numpy as np
import os
import pickle

In [ ]:
# Type of input data
data_type = "spec" # "eeg_raw" #"eeg_spec"  #
# number of subprocesses to use for data loading
import multiprocessing as cpu
num_workers = 0 #cpu.cpu_count() #- 1
# how many samples per batch to load
batch_size = 64

model_name =   "TransNet_Efficientnetb0" #"TransNet_Resnet18" #
input_shape =  (64,512) if model_name == "TransNet_Resnet18" else (64,1280)
batch_size = input_shape[0]
N_features = input_shape[1]

In [ ]:
# Load (train or test) data from csv file
if local:
    path = f"C:/Users/Amy/Desktop/Green_Git/eegClassification/"
else:
    path = f"./"


In [ ]:
text = 'features'
data_path = path+f'data/{model_name}_{text}_{data_type}/'
data_files = os.listdir(data_path)
data_features = [f for f in data_files if f.startswith(text)]
data_votes = [f for f in data_files if f.startswith('votes')]
N_items = len(data_features)

print("Number of items", N_items)

In [ ]:
np.load(data_path+data_features[0]).shape, np.load(data_path+data_votes[0]).shape

In [ ]:
# load all data_features into a single numpy array
X = np.zeros((N_items,*input_shape))
for i in range(N_items):
    X[i] = np.load(data_path+data_features[i])

# load all data_votes into a single numpy array
Y = np.zeros((N_items, batch_size, N_classes))
for i in range(N_items):
    Y[i] = np.load(data_path+data_votes[i])

In [ ]:
X.shape, Y.shape

In [ ]:
# reshape data to (N_items*input_shape[0], input_shape[1])
X = X.reshape(N_items*batch_size, N_features)
Y = Y.reshape(N_items*batch_size, N_classes)

In [ ]:
X.shape, Y.shape

In [ ]:
# convert votes to class probabilities
Y = Y/np.sum(Y, axis=1)[:,np.newaxis]

In [ ]:
# one hot encode the votes
Y_oh = np.argmax(Y, axis=1)
# Y_oh = np.eye(N_classes)[Y_oh]

In [ ]:
# xgboost for feature importance on a classification problem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y_oh, test_size=0.33, random_state=7)


In [ ]:
# 5 fold cross validation to find the best parameter configuration
from sklearn.model_selection import GridSearchCV, StratifiedKFold

objective = 'multi:softmax'
num_class=N_classes
model = XGBClassifier(objective=objective, 
                      num_class=num_class,
                      )

# grid search
n_estimators = [100, 200, 300]
learning_rate = [0.0001, 0.001, 0.01]
max_depth = [3, 6, 9]
param_grid = dict(learning_rate=learning_rate, 
                  n_estimators=n_estimators, 
                  max_depth=max_depth,)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']

In [ ]:
# train model with best parameters
model = XGBClassifier(objective=objective, num_class=num_class, **grid_result.best_params_)
model.fit(X_train, y_train)

In [ ]:
# save model to file
filename = f"./xgboost_{model_name}_{data_type}.sav"
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# make predictions for test data
y_pred = model.predict_proba(X_test)

# save predictions to file
filename = f"./xgboost_predictions_{model_name}_{data_type}.npy"
np.save(filename, y_pred)

In [ ]:
y_pred.shape, ', '.join(["{0:0.2f}".format(x) for x in y_pred[0]])

In [ ]:
predictions = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)

In [ ]:
predictions[0], y_test_class[0]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test_class, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

confusion_matrix = confusion_matrix(y_test_class, predictions)

plt.figure(figsize=(10, 10))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap='Blues')
plt.xticks(ticks=np.arange(6) + 0.5, labels=classes)
plt.yticks(ticks=np.arange(6) + 0.5, labels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
y_test[0], y_pred[0]

In [ ]:
# evaluate using kl divergence
from scipy.stats import entropy
kl_div = np.mean([entropy(y_test[i], y_pred[i]) for i in range(len(y_test))])
lk_base = np.mean([entropy(y_test[i], np.array([1/6]*6)) for i in range(len(y_test))])
print(f"KL div: {kl_div:.3f} (baseline: {lk_base:.3f})")